In [1]:
from n_body_system import NBodySystem
import torch
import numpy as np

# Create simulated data for training

## Create N-body system

In [2]:
dim = 2        # dimension
num_bodies = 5 # number of bodies

force_1_over_r = lambda r, G, m1, m2: G * m1 * m2 / (r + 1e-8)
system = NBodySystem(N=num_bodies, dimension=dim, G=1, R=1, force_law=force_1_over_r, seed=42)

## Run dynamics 

In [3]:
dt = 0.005 
num_steps = 1000    # number of time steps

# Preallocate arrays
nodes_t = np.zeros((num_steps, num_bodies, 2*dim+1))
vel_updated_true = np.zeros((num_steps, num_bodies, dim))

# Run simulation
for t in range(num_steps):
    nodes_t[t, :, 0:dim] = system.positions
    nodes_t[t, :, dim:2*dim] = system.velocities
    nodes_t[t, :, -1] = system.masses.flatten()

    system.step(dt)  # advance velocities

    vel_updated_true[t, :, :] = system.velocities.copy()

print("nodes_t shape:", nodes_t.shape)              # (num_steps, num_bodies, 2*dim+1)
print("vel_updated_true shape:", vel_updated_true.shape)  # (num_steps, num_bodies, dim)

nodes_t shape: (1000, 5, 5)
vel_updated_true shape: (1000, 5, 2)


## Create sender and receiver indices

In [4]:
# Create all pairs (i,j) where i != j
senders, receivers = np.meshgrid(np.arange(num_bodies), np.arange(num_bodies))
senders = senders.flatten()
receivers = receivers.flatten()

# Remove self-loops
mask = senders != receivers
senders = senders[mask]
receivers = receivers[mask]

# Combine into edge_index
edge_index = np.stack([senders, receivers], axis=0)  # shape: [2, N*(N-1)]
print(edge_index.shape)

(2, 20)


## Save data in torch files .pt

In [5]:
nodes_t_tensor = torch.tensor(nodes_t, dtype=torch.float32)
vel_updated_true_tensor = torch.tensor(vel_updated_true, dtype=torch.float32)
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)

torch.save(nodes_t_tensor, "training_data/nodes_t.pt")
torch.save(vel_updated_true_tensor, "training_data/vel_updated_true.pt")
torch.save(edge_index_tensor, "training_data/edge_index.pt")

print(edge_index_tensor.shape)

torch.Size([2, 20])


# Create simulated data for zero-shot generalization 

In [6]:
dim2 = 2        # dimension
num_bodies2 = 7 # number of bodies

force_1_over_r = lambda r, G, m1, m2: G * m1 * m2 / (r + 1e-8)
system2 = NBodySystem(N=num_bodies2, dimension=dim2, G=1, R=1, force_law=force_1_over_r, seed=42)

In [7]:
dt2 = 0.005 
num_steps2 = 100    # number of time steps

# Preallocate arrays
nodes_t2 = np.zeros((num_steps2, num_bodies2, 2*dim2+1))
vel_updated_true2 = np.zeros((num_steps2, num_bodies2, dim2))

# Run simulation
for t in range(num_steps2):
    nodes_t2[t, :, 0:dim2] = system2.positions
    nodes_t2[t, :, dim2:2*dim2] = system2.velocities
    nodes_t2[t, :, -1] = system2.masses.flatten()

    system2.step(dt2)  # advance velocities

    vel_updated_true2[t, :, :] = system2.velocities.copy()

print("nodes_t shape:", nodes_t2.shape)              # (num_steps, num_bodies, 2*dim+1)
print("vel_updated_true shape:", vel_updated_true2.shape)  # (num_steps, num_bodies, dim)

nodes_t shape: (100, 7, 5)
vel_updated_true shape: (100, 7, 2)


In [8]:
# Create all pairs (i,j) where i != j
senders2, receivers2 = np.meshgrid(np.arange(num_bodies2), np.arange(num_bodies2))
senders2 = senders2.flatten()
receivers2 = receivers2.flatten()

# Remove self-loops
mask2 = senders2 != receivers2
senders2 = senders2[mask2]
receivers2 = receivers2[mask2]

# Combine into edge_index
edge_index2 = np.stack([senders2, receivers2], axis=0)  # shape: [2, N*(N-1)]
print(edge_index2.shape)

(2, 42)


In [9]:
nodes_t_tensor2 = torch.tensor(nodes_t2, dtype=torch.float32)
vel_updated_true_tensor2 = torch.tensor(vel_updated_true2, dtype=torch.float32)
edge_index_tensor2 = torch.tensor(edge_index2, dtype=torch.long)

torch.save(nodes_t_tensor2, "generalization_data/nodes_t.pt")
torch.save(vel_updated_true_tensor2, "generalization_data/vel_updated_true.pt")
torch.save(edge_index_tensor2, "generalization_data/edge_index.pt")